# Observations

### Input file format
- Each file represents one image from OIDv4, the filename is the same as the image name
- One text file can have multiple bounding boxes
- The coordinates are in the format as follows (class name, left upper corner (x), left upper corner (y), right lower corner (x), right lower corner (y))

In [1]:
!cat oid-samples/2558dd5cb4424643.txt

French fries 80.743424 110.21311999999999 952.539136 933.18656


In [7]:
!cat oid-samples/2c48da973b97f97d.txt

French fries 0.003072 94.44096 390.41024 380.403456
French fries 180.13183999999998 254.459136 890.2789119999999 656.116992


### Output file format 
- File is in XML format
- folder is class name
- path is arbituary, but should have the filename
- size has dimensions
- multiple objects can be represented by <object> tags
- object has a different value expectation than above, it represent the edges of the bounding box, rather than coordinates of top left and bottom right corners, it also is an integer rather than floating point

In [8]:
!cat output-format/20190220_160312.xml

<annotation>
	<folder>goat</folder>
	<filename>20190220_160312.jpg</filename>
	<path>C:\Users\romolo.rocchi\Desktop\goat\20190220_160312.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>640</width>
		<height>480</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>goat</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>102</xmin>
			<ymin>186</ymin>
			<xmax>226</xmax>
			<ymax>263</ymax>
		</bndbox>
	</object>
	<object>
		<name>goat</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>436</xmin>
			<ymin>186</ymin>
			<xmax>567</xmax>
			<ymax>299</ymax>
		</bndbox>
	</object>
</annotation>


# Creating the template
Need to have two templates, master and object level (bounding box level)

In [ ]:
master_template = """
<annotation>
    <folder>{class_name}</folder>
    <filename>{filename}</filename>
    <path>/path/to/the/file/{filename}</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>{image_width}</width>
        <height>{image_height}</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    { objects }
</annotation>
"""

In [ ]:
object_template ="""<object>
        <name>{class_name}</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{xmin}</xmin>
            <ymin>{186}</ymin>
            <xmax>226</xmax>
            <ymax>263</ymax>
        </bndbox>
</object>
"""